In [60]:
import pandas as pd
import re

In [61]:
df_all = pd.read_table(
    'Dmel_gene.gff',header=None,
    names=['seqname','source','feature','start','end','score','strand','frame','attribute']
)

In [62]:
df_selected = pd.read_csv('genes.csv')

In [63]:
df_range = df_selected['cluster region'].str.split(':|-',expand=True)
df_range = df_range.rename(columns={0:'chr',1:'start',2:'end'})
df_range = df_range.dropna()
df_range['start'] = df_range['start'].astype(int)
df_range['end'] = df_range['end'].astype(int)

In [99]:
df_all['attribute'][1]

'ID=gene-Dmel_CR40469;Dbxref=FLYBASE:FBgn0058469,GeneID:5740321;Name=lncRNA:CR40469;cyt_map=1A1-1A1;description=long non-coding RNA:CR40469;gbkey=Gene;gen_map=1-0 cM;gene=lncRNA:CR40469;gene_biotype=lncRNA;gene_synonym=CR40469,Dmel\\CR40469;locus_tag=Dmel_CR40469'

In [65]:
Chr = None
df_negative_control = pd.DataFrame()
for i,row in df_all.iterrows():
    print(i,end='\r',flush=True)
    flag = False
    if row['feature'] == 'region':
        Chr = re.search(r'chromosome=(.*?);',row['attribute']).group(1)
        continue
    else:
        start = row['start']
        end = row['end']
        for j,row2 in df_range.iterrows():
            if (row2['chr'] == Chr) and \
            ((row2['start'] < max(start,end)) and (row2['end'] > min(start,end))):
                flag = True
                break
        if not flag:
            df_negative_control = df_negative_control.append(row)

In [ ]:
biotype = re.search(r'gene_biotype=(.*?);',row['attribute']).group(1)

In [92]:
df_negative_control['FLYBASE'] = df_negative_control['attribute'].str.extract(r'FLYBASE:(.*?),')

In [100]:
df_negative_control['biotype'] = \
    df_negative_control['attribute'].str.extract(r'gene_biotype=(.*?);')

In [102]:
df_negative_control.loc[df_negative_control['biotype'] == 'protein_coding','FLYBASE']

2        FBgn0025837
3        FBgn0025836
5        FBgn0284435
7        FBgn0025835
9        FBgn0040373
            ...     
17467    FBgn0267417
17468    FBgn0267491
17469    FBgn0046698
17470    FBgn0046323
17471    FBgn0267592
Name: FLYBASE, Length: 4354, dtype: object

In [104]:
df_negative_control.loc[df_negative_control['biotype'] == 'protein_coding','FLYBASE'].to_csv(
    'FB_names_negative_all.csv',index=False)

In [105]:
df_negative_control.loc[df_negative_control['biotype'] == 'protein_coding','FLYBASE'].sample(
    n=114).to_csv('FB_names_negative.csv',index=False)